In [1]:
#initial imports
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import numpy as np

In [2]:
#read in race incidence csv from github

incidence_df = pd.read_csv("https://raw.githubusercontent.com/robyndook/Cancer_Treatment_Centers_California/main/Resources/Race_Incidence_Cancer_DB.csv")                         
#get rid of commas in columns
incidence_df = incidence_df.replace(',','', regex=True)
incidence_df.head()     

,average_annual_count,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,county_state,index_of_medical_underservice_score,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019,abbrv,median_household_income_2015,median_household_income_2016,median_household_income_2017,median_household_income_2018,median_household_income_2019
0,30,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,681.2,Lowndes County AL,54.0,10350,10248,10097,9974,9726,AL,31117.0,32011.0,33130.0,33973.0,33930.0
1,30,Alabama,All Cancer Sites,White Non-Hispanic,All Sexes,All Ages,694.0,Lowndes County AL,54.0,10350,10248,10097,9974,9726,AL,31117.0,32011.0,33130.0,33973.0,33930.0
2,51,Alabama,All Cancer Sites,Black (includes Hispanic),All Sexes,All Ages,570.1,Lowndes County AL,54.0,10350,10248,10097,9974,9726,AL,31117.0,32011.0,33130.0,33973.0,33930.0
3,80,Alabama,All Cancer Sites,White (includes Hispanic),All Sexes,All Ages,558.3,Crenshaw County AL,58.0,13850,13916,13858,13825,13772,AL,36759.0,37374.0,38419.0,39057.0,43309.0
4,80,Alabama,All Cancer Sites,White Non-Hispanic,All Sexes,All Ages,565.3,Crenshaw County AL,58.0,13850,13916,13858,13825,13772,AL,36759.0,37374.0,38419.0,39057.0,43309.0


In [3]:
#get count of rows
incidence_df.count()

average_annual_count                   9160
primary_state_name                     9160
cancer                                 9160
race_ethnicity                         9160
sex                                    9160
age                                    9160
age_adjusted_rate                      9160
county_state                           9160
index_of_medical_underservice_score    7931
popestimate2015                        9160
popestimate2016                        9160
popestimate2017                        9160
popestimate2018                        9160
popestimate2019                        9160
abbrv                                  9146
median_household_income_2015           9146
median_household_income_2016           9146
median_household_income_2017           9146
median_household_income_2018           9146
median_household_income_2019           9146
dtype: int64

In [4]:
#get count of values for race
incidence_df["race_ethnicity"].value_counts()

White (includes Hispanic)                       2643
  White Non-Hispanic                            2335
Black (includes Hispanic)                       1308
Hispanic (any race)                             1009
  White Hispanic                                 912
Asian / Pacific Islander (includes Hispanic)     525
Amer. Indian / AK Native (includes Hispanic)     428
Name: race_ethnicity, dtype: int64

In [5]:
#clean race values
incidence_df['race_ethnicity'] = incidence_df['race_ethnicity'].str.strip()

In [6]:
#get count of values for race
incidence_df["race_ethnicity"].value_counts()

White (includes Hispanic)                       2643
White Non-Hispanic                              2335
Black (includes Hispanic)                       1308
Hispanic (any race)                             1009
White Hispanic                                   912
Asian / Pacific Islander (includes Hispanic)     525
Amer. Indian / AK Native (includes Hispanic)     428
Name: race_ethnicity, dtype: int64

In [7]:
#drop rows where race equals white non-hispanic and race equals white hispanic to avoid overlap
incidence_df = incidence_df[incidence_df["race_ethnicity"] != "White Non-Hispanic"]
incidence_df = incidence_df[incidence_df["race_ethnicity"] != "White Hispanic"]

In [8]:
#get count of values for race
incidence_df["race_ethnicity"].value_counts()

White (includes Hispanic)                       2643
Black (includes Hispanic)                       1308
Hispanic (any race)                             1009
Asian / Pacific Islander (includes Hispanic)     525
Amer. Indian / AK Native (includes Hispanic)     428
Name: race_ethnicity, dtype: int64

In [9]:
#get count of rows
incidence_df.count()

average_annual_count                   5913
primary_state_name                     5913
cancer                                 5913
race_ethnicity                         5913
sex                                    5913
age                                    5913
age_adjusted_rate                      5913
county_state                           5913
index_of_medical_underservice_score    5165
popestimate2015                        5913
popestimate2016                        5913
popestimate2017                        5913
popestimate2018                        5913
popestimate2019                        5913
abbrv                                  5905
median_household_income_2015           5905
median_household_income_2016           5905
median_household_income_2017           5905
median_household_income_2018           5905
median_household_income_2019           5905
dtype: int64

In [10]:
#create target 
y = incidence_df[['age_adjusted_rate']]

#change dtype to numeric
y['age_adjusted_rate'] = y['age_adjusted_rate'].astype(float)
y.head()

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_adjusted_rate
0,681.2
2,570.1
3,558.3
5,442.0
6,542.5


In [11]:
#create feautures variable holding race
X = incidence_df[["race_ethnicity"]]
X.head()

,race_ethnicity
0,White (includes Hispanic)
2,Black (includes Hispanic)
3,White (includes Hispanic)
5,Black (includes Hispanic)
6,White (includes Hispanic)


In [12]:
#create dummy variables for race
X = pd.get_dummies(X, columns=["race_ethnicity"])
X.head()

,race_ethnicity_Amer. Indian / AK Native (includes Hispanic),race_ethnicity_Asian / Pacific Islander (includes Hispanic),race_ethnicity_Black (includes Hispanic),race_ethnicity_Hispanic (any race),race_ethnicity_White (includes Hispanic)
0,0,0,0,0,1
2,0,0,1,0,0
3,0,0,0,0,1
5,0,0,1,0,0
6,0,0,0,0,1


In [13]:
#get shape of target array
y.shape

(5913, 1)

In [14]:
#get shape of feautures array
X.shape

(5913, 5)

In [15]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [16]:
#create linear regression model using sklearn
model = LinearRegression()

In [17]:
#fit the model
regression_race_incidence = model.fit(X_train ,y_train)

In [19]:
#Create predictions array using linear regression
y_pred = regression_race_incidence.predict(X_test)
y_pred

array([[450.25],
       [458.25],
       [450.25],
       ...,
       [458.25],
       [347.75],
       [450.25]])

In [20]:
#get R2 value from linear regression
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.19591440986630704

In [21]:
#run regression again with statsmodels
#use statsmodels to add constant to X
#add constant to X
X1 = sm.add_constant(X)

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [22]:
#fit linear regression model
model = sm.OLS(y, X1).fit()

In [23]:
#view model summary with statsmodels
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      age_adjusted_rate   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     393.7
Date:                Fri, 03 Jun 2022   Prob (F-statistic):          4.47e-301
Time:                        14:54:19   Log-Likelihood:                -36246.
No. Observations:                5913   AIC:                         7.250e+04
Df Residuals:                    5908   BIC:                         7.253e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [24]:
##decision tree model
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [25]:
#fit the model with training data
regressor_DT = DecisionTreeRegressor()
regressor_DT.fit(X_train, y_train)

DecisionTreeRegressor()

In [26]:
#predict
y_pred = regressor_DT.predict(X)
y_pred

array([451.93381148, 457.65481928, 451.93381148, ..., 346.82545455,
       451.93381148, 451.93381148])

In [27]:

#calculate r2 score
r2_score(y, y_pred)

0.21022658462225763